In [1]:
%reload_ext tensorboard
%tensorboard --logdir ./Project_Files/TimeSeries_Prediction/darts_logs --host localhost --port 8999

In [2]:
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)

In [3]:
from helper_funcs.preprocessing import total_timeseries, get_covariates
from helper_funcs.prediction import historical_predictions, total_validation
from helper_funcs.inverse import inverse_func

from darts.utils.model_selection import train_test_split
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from darts.models import NLinearModel, TFTModel, NaiveSeasonal, NHiTSModel

import torch
import eco2ai

from models import nlinear, tft, nhits

RANDOM = 101
INPUT_CHUNK = 15
OUTPUT_CHUNK = 5
TEST_SIZE = 0.2
RETRAIN=True
LAST=False
RESET=False
EXP_MA = 15
SAVE = True
EPOCHS = 100

checkpoint = ModelCheckpoint(monitor="val_loss")
progress_bar = RichProgressBar()
tracker = eco2ai.Tracker(project_name="Stock_Market_Prediction", experiment_description="training multiple models")

/home/andrei/miniconda3/envs/hse/lib/python3.10/site-packages/eco2ai/emission_track.py:135: UserWarning: 
If you use a VPN, you may have problems with identifying your country by IP.
It is recommended to disable VPN or
manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes

  warnings.warn(


In [4]:
callbacks=[progress_bar]

In [5]:
timeseries = total_timeseries(EXP_MA, market=True, sentiment=True, embeddings=True)
timeseries = StaticCovariatesTransformer().fit_transform(timeseries)

In [6]:
train, val = train_test_split(
    timeseries,
    axis=1,
    test_size=TEST_SIZE,
    input_size=INPUT_CHUNK,
    horizon=OUTPUT_CHUNK,
    vertical_split_type='model-aware'
)

data = [train, val]

target_train, past_train, future_train, target_val, past_val, future_val, target_test, past_test, future_test = get_covariates(
    data,
    target='Close',
    past_covariates=[
        # 'Adj Close',
        'High',
        'Low',
        'Open',
        'Volume',
        # 'Volatility',
        # 'Negative',
        # 'Positive',
        'sentiment_score_1',
        # 'sentiment_score_2'
        ],
    embeddings=False
    )

name = 'model_tft_HHLOVS1'

In [7]:
past_train[0].components

Index(['High', 'Low', 'Open', 'Volume', 'sentiment_score_1'], dtype='object', name='component')

In [8]:
scaler_target = Scaler()
scaler_covariates = Scaler()

In [9]:
target_train_scaled = scaler_target.fit_transform(target_train)
target_val_scaled = scaler_target.transform(target_val)
target_test_scaled = scaler_target.transform(target_test)


past_train_scaled = scaler_covariates.fit_transform(past_train)
past_val_scaled = scaler_covariates.transform(past_val)
past_test_scaled = scaler_covariates.transform(past_test)

In [10]:
# baseline = NaiveSeasonal(K=5)

# baseline_hist = historical_predictions(
#     baseline, target_val_scaled, INPUT_CHUNK, OUTPUT_CHUNK, RETRAIN, LAST, \
#         save=SAVE, past=past_val_scaled, future=future_val)

In [11]:
tracker.start()

if name.split('_')[1] == 'nlinear':
    model = nlinear.nlinear_emb(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
        callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
            future_train, future_val, EPOCHS)
            
    model = NLinearModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

if name.split('_')[1] == 'tft':
    model = tft.tft_set(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
        callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
            future_train, future_val, EPOCHS)
    
    model = TFTModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

if name.split('_')[1] == 'nhits':
    model = nhits.nhits(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
        callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
            future_train, future_val, EPOCHS)
            
    model = NHiTSModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

In [ ]:
hist = historical_predictions(
    model, target_val_scaled, INPUT_CHUNK, OUTPUT_CHUNK, RETRAIN, LAST, \
        save=SAVE, past=past_val_scaled, future=future_val)

tracker.stop()